We will do the following steps in order:

1. Load and normalize the CIFAR10 training and test datasets using torchvision
2. Load Pytorch model
3. Test the network on the test data
4. Convert model to ONNX and OpenVINO IR format
5. Evaluate converted model accuracy
6. Optimize model to INT8 precision

### 1. Load and normalize CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# define preprocessing steps for model
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

batch_size = 4

# define data loaders for training and validation
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

# categories used in dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 2. Load Pytorch model

In [2]:
import torch

model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)

Using cache found in C:\Users\eaidova/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


### 3. Test the network on the test data

We have trained the network and need to check how well it performes on valikdation datat.

We will check this by predicting the class label that the neural network outputs, and checking it against the ground-truth. If the prediction is correct, we add the sample to the list of correct predictions.

now let us see what the neural network thinks these examples above are:

The outputs are energies for the 10 classes. The higher the energy for a class, the more the network thinks that the image is of the particular class. So, let’s get the index of the highest energy:

In [3]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 72.17 %


### 4. Convert model to ONNX and OpenVINO IR format

We want to run model with OpenVINO framework. For that we need to convert it to IR using OpenVINO Model Optimizer tool. The best practice for PyTorch models to export them to ONNX before starting work.

In [7]:
torch.onnx.export(model, images[0].unsqueeze(0), 'resnet20-cifar10.onnx')

In [8]:
!mo --input_model resnet20-cifar10.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	c:\Users\eaidova\repos\open_model_zoo\data\dataset_api\resnet20-cifar10.onnx
	- Path for generated IR: 	c:\Users\eaidova\repos\open_model_zoo\data\dataset_api\.
	- IR output name: 	resnet20-cifar10
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
	- Use legacy API for model processing: 	False
	- Use the transformations config file: 	None
	- OpenVINO runtime fo

### 5. Evaluate converted model accuracy

Now we want to check how model works with OpenVINO. We already have dataset and transformation steps defined for PyTorch and need to adopt to work with OpenVINO model.

In [3]:
from openvino.model_zoo.datasets import CIFAR10Dataset # is analog of data loader
from openvino.model_zoo.transforms.input import from_torch # translater for torchvision transforms to openvino

ov_transforms = from_torch(transform) # wrap our torchvision transforms
test_loader = CIFAR10Dataset('./data', reader='pillow_imread')

Annotation conversion for cifar10 dataset has been started
Parameters to be used for conversion:
converter: cifar
data_batch_file: data\cifar-10-batches-py\test_batch
convert_images: True
converted_images_dir: data\test
num_classes: 10
Annotation conversion for cifar10 dataset has been finished


The data_loader has similar to tochvision interface, it allow access to dataset annotation and input data using integer index and iterate over samples. Optional step is appling transformations to input data

The data in iteration represented in following format (batch_idx, batch_annotation), batch_input, batch_metadata, where:

* batch_id - index of selected samples from dataset for data batch
* batch_annotation - batch of labels for selected samples
* batch_input - input data
* batch_metadata - additional dictionary with useful meta for samples (it may contains e.g. info about dataset labels or preprocessing ops history for postprocessing)

Load model and get it predictions

In [4]:
from openvino.runtime import Core

core = Core()
model = core.read_model('resnet20-cifar10.xml')
compiled_model = core.compile_model(model, 'CPU')
output = compiled_model.outputs[0]
infer_request = compiled_model.create_infer_request()

In [5]:

test_loader.set_transforms([ov_transforms]) # set transformations for input data
test_loader.set_input_info(model.inputs) # set transforms for feeding to model inputs

Now lets calculate accuracy of converted network. For accuracy measurement we will use metrics.

In [6]:
from openvino.model_zoo.metrics import create_accuracy
from openvino.model_zoo.transforms.output import ClassificationPostprocessor

accuracy = create_accuracy()
postprocess_transform = ClassificationPostprocessor()

In [7]:
for idx, (batch_idx_batch_annotation, batch_input, batch_meta) in enumerate(test_loader):
    batch_ids, batch_annotations = map(list, zip(*batch_idx_batch_annotation))
    infer_result = infer_request.infer(batch_input[0])
    batch_predictions = postprocess_transform(infer_result, batch_meta)
    accuracy.batch_update(batch_annotations, batch_predictions)

In [7]:
print(f'Model accuracy: {accuracy.evaluate()}')

Model accuracy: 0.8746


As it can be seen, no accuracy degradation found during evaluation.
Now, let try to optimize model via Posttraining Optimization tool

In [8]:
import addict
from openvino.tools.pot import IEEngine, save_model, compress_model_weights, create_pipeline, load_model
from openvino.tools.pot.engines.utils import process_raw_output

# define configuration

algorithms = [
        {
            'name': 'DefaultQuantization',
            'params': {
                'target_device': 'ANY',
                'preset': 'performance',
                'stat_subset_size': 300
            }
        }
]

model_config = addict.Dict({
        'model_name': 'sample_model',
        'model': 'resnet20-cifar10.xml',
        'weights': 'resnet20-cifar10.bin'
})

engine_config = addict.Dict({
        'device': 'CPU',
        'stat_requests_number': 4,
        'eval_requests_number': 4
})

c:\Users\eaidova\Anaconda3\envs\py37\lib\site-packages\defusedxml\__init__.py:30: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  from . import cElementTree



In [10]:
class Engine(IEEngine):
    def _process_infer_output(self, stats_layout, predictions,
                              batch_annotations, batch_meta, need_metrics_per_sample):
        # Collect statistics
        if stats_layout:
            self._collect_statistics(outputs=predictions,
                                     stats_layout=stats_layout,
                                     annotations=batch_annotations)

        # Postprocess network output
        outputs = process_raw_output(predictions)
        postprocess_transform.output_tensor = str(self._output_layers[0])
        # Update metrics
        if batch_annotations:
            batch_predictions = postprocess_transform(outputs)

            self._update_metrics(batch_predictions, annotations=batch_annotations,
                                 need_metrics_per_sample=need_metrics_per_sample)

    def _update_metrics(self, output, annotations, need_metrics_per_sample=False):
        """ Updates metrics.
        :param output: network output
        :param annotations: a list of annotations for metrics collection [(img_id, annotation)]
        :param need_metrics_per_sample: whether to collect metrics for each batch
        """
        annotations_are_valid = all(a is not None for a in annotations)

        if self._metric and annotations_are_valid:
            sample_ids, batch_annotations = map(list, zip(*batch_idx_batch_annotation))
            metrics = self._metric.batch_update(batch_annotations, output)
            if need_metrics_per_sample:
                metrics = self._metric.value
                for sample_id, metric in zip(sample_ids, metrics):
                    for metric_name, metric_value in metric.items():
                        self._per_sample_metrics.append({'sample_id': sample_id,
                                                'metric_name': metric_name,
                                                'result': metric_value})

    def _process_batch(self, batch):
        _batch = batch[0]
        batch_annotation, batch_input, batch_meta = _batch[0], _batch[1], _batch[2]

        return batch_annotation, batch_input, batch_meta

In [11]:
engine = Engine(engine_config, test_loader, accuracy)
pipeline = create_pipeline(algorithms, engine)

In [12]:
model_rep = load_model(model_config)
compressed_model = pipeline.run(model_rep)

In [15]:
compress_model_weights(compressed_model)
save_model(compressed_model, 'optimized', 'resnet20-cifar10')

[{'model': 'optimized\\resnet20-cifar10.xml',
  'weights': 'optimized\\resnet20-cifar10.bin'}]

In [16]:
metric_results = pipeline.evaluate(compressed_model)


In [18]:
for key, value in metric_results.items():
    print(f'Quantized model {key}: {value * 100} %')

Quantized model accuracy: 9.36 %
